<a href="https://colab.research.google.com/github/anuppohekar/Deep-Learning-Projects/blob/main/Copy_of_SpeechRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget 'http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip'
!unzip mini_speech_commands.zip
!rm '/content/mini_speech_commands/README.md'

In [ ]:
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
import warnings

warnings.filterwarnings("ignore")

In [ ]:
path = '/content/mini_speech_commands/up/0132a06d_nohash_2.wav'
soundnumpy, _ = librosa.load(path, 
             sr=16000,
             mono=True,
             offset=0.0,
             duration=None, 
             dtype=np.float32, 
             res_type='kaiser_best')

In [ ]:
soundnumpy.shape

In [ ]:
plt.plot(soundnumpy)

#Create a dataset

In [ ]:
X = []
y = []

In [ ]:
folders = ['down','go','left','no','right','stop','up','yes']

In [ ]:
from tqdm import tqdm
import os 

path = '/content/mini_speech_commands'
for i in folders:
  for audio in tqdm(os.listdir(path+'/'+i)):
    filepath=os.path.join(path+'/'+i+'/'+audio)
    samples, _ = librosa.load(path=filepath, sr=16000)
    if(len(samples)==16000):
      X.append(samples)
      y.append(i)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

X = np.array(X)
y = np.array(y)
le = LabelEncoder()
y = le.fit_transform(y)
y = np_utils.to_categorical(y, 8)

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units=10000, activation='relu', input_dim=16000))
model.add(Dense(units=8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)
filepath='bestweights.h5'
mc = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True)
rd = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto')

In [ ]:
model.fit(x=X_train, y=y_train, epochs=10, validation_split=0.2)